> 大家好，我是卢冠鹏，本次案例通过使用北京的链家二手房数据库，研究了房型结构/房龄/面积/城郊时间的集中趋势以及对成交价格的影响。并引入机器学习的决策树算法来预测房价，进行模型评估以及校准，以得到对于北京房价的最佳预测。

> 首先载入本次分析用到的Python模组（Pandas,Numpy,sklearn,seaborn等）:

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
print ("Set up complete")
# Any results you write to the current directory are saved as output.

> 1.数据库导入

In [ ]:
df = pd.read_csv("../input/lianjia/new.csv",encoding='gbk')

> 初步观察数据库的大致分布情况:

In [ ]:
df.describe()

> 2.数据清洗

> 观察每列的数据类型是否错误:

In [ ]:
df.dtypes

> 观察各列的数据类型发现'tradeTime','livingRoom','drawingRoom','bathRoom','floor','constructionTime'数据类型有错误,需要首先进行处理:

In [ ]:
#提取交易年份单独作为一列:
df['tradeyear']=df.tradeTime.astype(str).str[0:4]

In [ ]:
#去除livingRoom中的异常数值，更改数据类型
df=df.loc[df.livingRoom!='#NAME?',:]
df.livingRoom=df.livingRoom.astype('int64')
df.livingRoom.value_counts()

In [ ]:
#统一drawingRoom,bathRoom的数据为整数
df.drawingRoom=df.drawingRoom.astype('int64')
df.bathRoom=df.bathRoom.astype('int64')

In [ ]:
#将floor列中的汉子去除，保留楼层数为整数
df.floor=df.floor.apply(lambda df:df.split()[1])
df.floor=df.floor.astype('int64')

In [ ]:
#将'constructionTime'列中的未知项跳过,只考虑1950到2016年建成的房子:
df.constructionTime=df.constructionTime.replace('未知','9999').astype('int64')
df=df.loc[(df.constructionTime>=1950)&(df.constructionTime<=2016),:]

In [ ]:
df.head()

> 3.北京历年的均价走势:

> 在其它分析开始前，首先整体看一下北京市二手市场的历年整体均价变化情况:

In [ ]:
df_year=df.groupby('tradeyear').price.mean()
sns.lineplot(data=df_year)

> 由此图可以看到北京市均价在2010年前后为最低，随后一路上涨，2017年达到高峰，随后向下小幅波动。

> 4.地段对房子交易价格的影响

> 对房价的重要影响因素之一是房子所处地段，此数据库包含交易房子的经纬坐标，通过将地理坐标可视化，可以直观看到地段对房价的影响:

In [ ]:
#看一下数据库中历年交易记录数量，发现2016年的交易记录最为齐全
df.tradeyear.value_counts()

In [ ]:
df_2016=df.loc[df.tradeyear=='2016',:]

In [ ]:
plt.figure(figsize=(5,5))
sns.scatterplot(x=df_2016['Lng'],y=df_2016['Lat'],hue=df_2016['price'])

> 可以看到北京核心区域的分布情况（以天安门为中心向外辐射），集中区域均价超过十万。

> 5.房型，房龄，面积，装修情况的分布情况

In [ ]:
sns.distplot(a=df_2016['square'],kde=False)

In [ ]:
sns.scatterplot(x=df_2016['square'],y=df_2016['followers'])

> 可以看到市售的二手房以50~100平方的小户型居多，占据总量大半。而且收到最高关注的房子也都是在1~100平方的户型内分布。

> 进一步看一下卧室/洗手间的数量分布:

In [ ]:
sns.distplot(a=df_2016['livingRoom'],kde=False,label='livingRoom_Num')
sns.distplot(a=df_2016['bathRoom'],kde=False,label='bathRoom_Num')
plt.xlabel('Room_Num')
plt.ylabel('Qty')
plt.legend()

> 可以看到两卧/一卫的户型数量最多，卧室数量基本为1~3室。

In [ ]:
livingRoom_Num=df_2016.groupby('livingRoom').price.mean()
sns.barplot(x=livingRoom_Num.index,y=livingRoom_Num)

> 通过比较不同卧室数量房子的成交均价，发现1~3卧的均价略高于其它户型。

In [ ]:
sns.distplot(a=df_2016['constructionTime'],kde=False)

> 房子的建造年份主要分布在1980年前后至今。

In [ ]:
plt.figure(figsize=(30,3))
ConstructionTime_price=df_2016.groupby('constructionTime').price.mean()
sns.barplot(x=ConstructionTime_price.index,y=ConstructionTime_price)

> 通过逐年比较不同房龄的成交均价（建成年份为横轴），发现房子的均价与房龄并无太大关联，甚至有高房龄伴随高售价的情况，分析可能是由于北京市历史悠久的房子尤为稀缺所致（如上图所示）

> 5.北京房价预测:

> 本次房价预测的模型基于决策树算法，并基于算法拟合结果进行调试，使拟合结果更加精确:

> 讲数据库分为算法训练样本和拟合检测样本:

In [ ]:
df_2016_filtered=df_2016.dropna(axis=0)
y=df_2016_filtered.price
Predict_features=['Lng','Lat','Cid','livingRoom','bathRoom','constructionTime','buildingType','renovationCondition','subway']
X=df_2016_filtered[Predict_features]
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = 0)

> 为避免出现算法过度拟合或拟合不足的情况，定义算法校验决策树模型，已找到最佳算法参数:

In [ ]:
def get_mae(max_leaf_nodes,train_X,train_y):
    model=DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes,random_state=0)
    model.fit(train_X,train_y)
    pred_val=model.predict(val_X)
    mae=mean_absolute_error(pred_val,val_y)
    return(mae)

In [ ]:
for max_leaf_nodes in [500,1000,2000,3000,5000]:
    my_mae=get_mae(max_leaf_nodes,train_X,train_y)
    print("Max_leaf_nodes:%d \t Mean absolute error:%d"%(max_leaf_nodes,my_mae))

> 由上结果可见，当决策树节点数量为1000时，拟合结果误差最小:
最终结果拟合如下:

In [ ]:
final_model=DecisionTreeRegressor(max_leaf_nodes=1000,random_state=1)
final_model.fit(X,y)
final_pred=final_model.predict(X)
final_mae=mean_absolute_error(final_pred,y)
print(final_mae)

In [ ]:
根据此算法，平均预测得的房价与实际2016年房价误差平均为6794元。